In [1]:
import mysql.connector
import getpass
import pandas as pd

import os
import shutil
try:
    import git
except ImportError:
    print("GitPython is nZot installed. Installing...")
    !pip install gitpython
    import git
    
    #I get warning messages with pymysql
import warnings
warnings.filterwarnings('ignore')

In [2]:
#cnx = mysql.connector.connect(host="localhost",user="root",password=getpass.getpass('Enter password: '), database='sakila')
cnx = mysql.connector.connect(host="localhost",user="root",password=getpass.getpass('Enter password: '), database="senate_0424")

Enter password: ········


In [4]:
def sql(query):
    cur = cnx.cursor(buffered=True)
    cur.execute(query)
    try:
        for x in cur:
            print(x)
    except TypeError as e:
        print("SUCCESS: but bro there is nothing to print", e)

In [5]:
def sql2(query):
    cur = cnx.cursor(buffered=True)
    cur.execute(query)
    try:
        for x in cur:
            print(x)
    except TypeError as e:
        print("SUCCESS: but bro there is nothing to print", e)
        
    return cur

In [6]:
def todf(query):
    df = pd.read_sql(query, cnx)
    return df

In [7]:
sql('SHOW TABLES')

('election',)
('finances',)
('results',)


In [8]:
#sql('DROP DATABASE results;')
#sql('CREATE DATABASE IF NOT EXISTS senate_0424')
#sql('SHOW DATABASES')

# results

In [9]:
results_df = pd.read_excel("senate_candidates_results_trimmed.xlsx")
results_df['gender'] = results_df['gender'].replace({'m': 1,'m ': 1, 'f': 0})
results_df.rename(columns={'gender': 'is_male'}, inplace=True)
print("Unique values in 'is_male' column:", results_df['is_male'].unique())

Unique values in 'is_male' column: [1 0]


In [10]:
def create_results_table():
    query = """
    CREATE TABLE IF NOT EXISTS results (
        year YEAR,
        state CHAR(2),
        fec_id CHAR(9),
        incumbent CHAR(1),
        first_name VARCHAR(255),
        last_name VARCHAR(255),
        party ENUM('R', 'D', 'L', 'O'),
        votes INT,
        vote_share FLOAT,
        total_votes INT,
        is_male CHAR(1),
        white CHAR(1),
        black CHAR(1),
        age INT,
        PRIMARY KEY (year, fec_id)
    )
    """
    cur = cnx.cursor()
    cur.execute(query)
    cnx.commit()
    print("Table 'results' has been created successfully.")

In [11]:
print('''
before''')
sql('SHOW TABLES')
sql("DROP TABLE IF EXISTS results")
print('table deleted')
sql('SHOW TABLES')
create_results_table()
print('''after''')
a = sql('SHOW TABLES')

print('''

Below are the columns for that new table:
      ''')
sql("SELECT column_name FROM information_schema.columns WHERE table_name = 'results'")


before
('election',)
('finances',)
('results',)
SUCCESS: but bro there is nothing to print 'NoneType' object is not subscriptable
table deleted
('election',)
('finances',)
Table 'results' has been created successfully.
after
('election',)
('finances',)
('results',)


Below are the columns for that new table:
      
('age',)
('black',)
('fec_id',)
('first_name',)
('incumbent',)
('is_male',)
('last_name',)
('party',)
('state',)
('total_votes',)
('vote_share',)
('votes',)
('white',)
('year',)


In [12]:
def insert_results_data(results_df):
    insert_query = """
    INSERT INTO results (year, state, fec_id, incumbent, first_name, last_name, party, votes, vote_share, total_votes, is_male, white, black, age)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    data = results_df.values.tolist()
    cur = cnx.cursor()
    cur.executemany(insert_query, data)
    cnx.commit()
    print("Data inserted into 'results' table successfully.")

In [13]:
insert_results_data(results_df)

Data inserted into 'results' table successfully.


In [14]:
results_sql_df = todf("SELECT * FROM results")
results_sql_df

,year,state,fec_id,incumbent,first_name,last_name,party,votes,vote_share,total_votes,is_male,white,black,age
0,2016,AK,S0AK00121,0,Joe,Miller,L,90825,0.292,311441,1,1,0,48
1,2016,AR,S0AR00150,1,John,Boozman,R,661984,0.598,1107522,1,1,0,65
2,2016,CO,S0CO00211,1,Michael,Bennet,D,1370710,0.500,2743029,1,1,0,51
3,2016,CT,S0CT00177,1,Richard,Blumenthal,D,920766,0.577,1596276,1,1,0,70
4,2016,CT,S0CT00250,0,Jeffery,Russell,O,16713,0.010,1596276,1,1,0,56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,2020,MN,S8MN00578,1,Tina,Smith,O,1566522,0.487,3214256,0,1,0,62
271,2020,MS,S8MS00261,1,Cindy,Hyde-Smith,R,709511,0.541,1311354,0,1,0,60
272,2020,MS,S8MS00287,0,Mike,Espy,D,578691,0.441,1311354,1,0,1,66
273,2020,NE,S8NE00216,0,Chris,Janicek,D,227191,0.244,930012,1,1,0,50


# finances

In [15]:
finances_df = pd.read_excel("senate_finances.xlsx")
finances_df.isna().sum(0)

year              0
committee_id      0
fec_id            0
committee_name    0
total_raised      0
total_spent       0
cash_start        0
cash_end          0
dtype: int64

In [16]:
def create_finances_table():
    query = """
    CREATE TABLE IF NOT EXISTS finances (
        year YEAR,
        committee_id CHAR(9),
        fec_id CHAR(9),
        committee_name VARCHAR(255),
        total_raised FLOAT,
        total_spent FLOAT,
        cash_start FLOAT,
        cash_end FLOAT,
        PRIMARY KEY (year, committee_id)
    )
    """
    cur = cnx.cursor()
    cur.execute(query)
    cnx.commit()
    print("Table 'finances' has been created successfully.")

In [17]:
print('before')
sql('SHOW TABLES')
sql("DROP TABLE IF EXISTS finances")
print('table deleted')
sql('SHOW TABLES')
create_finances_table()
print('after')
a = sql('SHOW TABLES')

print('''

Below are the columns for that new table:
      ''')
sql("SELECT column_name FROM information_schema.columns WHERE table_name = 'finances'")

before
('election',)
('finances',)
('results',)
SUCCESS: but bro there is nothing to print 'NoneType' object is not subscriptable
table deleted
('election',)
('results',)
Table 'finances' has been created successfully.
after
('election',)
('finances',)
('results',)


Below are the columns for that new table:
      
('cash_end',)
('cash_start',)
('committee_id',)
('committee_name',)
('fec_id',)
('total_raised',)
('total_spent',)
('year',)


In [18]:
def insert_finances_data(finances_df):
    insert_query = """
    INSERT INTO finances (year, committee_id, fec_id, committee_name, total_raised, total_spent, cash_start, cash_end)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """
    data = finances_df.values.tolist()
    cur = cnx.cursor()
    cur.executemany(insert_query, data)
    cnx.commit()
    print("Data inserted into 'finances' table successfully.")

In [19]:
insert_finances_data(finances_df)

Data inserted into 'finances' table successfully.


In [20]:
finances_sql_df = todf("SELECT * FROM finances")
finances_sql_df

,year,committee_id,fec_id,committee_name,total_raised,total_spent,cash_start,cash_end
0,2016,C00148999,S6IL00151,FRIENDS OF DICK DURBIN,1151490.0,1073760.0,741450.0,819176.00
1,2016,C00193342,S2KY00012,MCCONNELL SENATE COMMITTEE,2154060.0,1476860.0,248241.0,925444.00
2,2016,C00193623,S6AL00013,SHELBY FOR U S SENATE,3467640.0,11531400.0,18055400.0,9991640.00
3,2016,C00196774,S4MA00028,MARKEY COMMITTEE; THE,601552.0,803795.0,2227330.0,2025090.00
4,2016,C00230482,S0IA00028,GRASSLEY COMMITTEE INC,8954940.0,9787430.0,1838650.0,1006160.00
...,...,...,...,...,...,...,...,...
705,2020,C00752741,S4GA11285,RISCH PERDUE VICTORY COMMITTEE,0.0,0.0,0.0,0.00
706,2020,C00753590,S6LA00441,BERYL BILLIOT CAMPAIGN FUND,6374.6,6374.6,0.0,0.00
707,2020,C00753657,S4IA00194,RICK STEWART,8820.0,16092.8,0.0,-7272.83
708,2020,C00754770,S6SC04155,BILL BLEDSOE ELECTION COMMITTEE,0.0,0.0,0.0,0.00


# demographics

In [21]:
demographics_df = pd.read_excel("state_demographics.xlsx")
demographics_df.isna().sum(0)

year             0
state_name       0
total_pop        0
median_age       0
white_pop        0
black_pop        0
male             0
female           0
high_school      0
bach             0
born_in_state    0
employment       0
inflation        0
dtype: int64

In [22]:
def create_demographics_table():
    query = """
    CREATE TABLE IF NOT EXISTS demographics (
        year YEAR,
        state CHAR(2),
        total_pop INT,
        median_age INT,
        white_pop INT,
        black_pop INT,
        male INT,
        female INT,
        high_school INT,
        bach INT,
        born_in_state INT,
        employment INT,
        inflation INT,
        PRIMARY KEY (state, year)
    )
    """
    cur = cnx.cursor()
    cur.execute(query)
    cnx.commit()
    print("Table 'demographics' has been created successfully.")

In [23]:
print('before')
sql('SHOW TABLES')
sql("DROP TABLE IF EXISTS demographics")
print('table deleted')
sql('SHOW TABLES')
create_demographics_table()
print('after')
a = sql('SHOW TABLES')

print('''

Below are the columns for that new table:
      ''')
sql("SELECT column_name FROM information_schema.columns WHERE table_name = 'demographics'")

before
('election',)
('finances',)
('results',)
SUCCESS: but bro there is nothing to print 'NoneType' object is not subscriptable
table deleted
('election',)
('finances',)
('results',)
Table 'demographics' has been created successfully.
after
('demographics',)
('election',)
('finances',)
('results',)


Below are the columns for that new table:
      
('bach',)
('black_pop',)
('born_in_state',)
('employment',)
('female',)
('high_school',)
('inflation',)
('male',)
('median_age',)
('state',)
('total_pop',)
('white_pop',)
('year',)


In [24]:
def insert_demographics_data(demographics_df):
    insert_query = """
    INSERT INTO demographics (year, state, total_pop, median_age, white_pop, black_pop, male, female, high_school, bach, born_in_state, employment, inflation)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    data = demographics_df.values.tolist()
    cur = cnx.cursor()
    cur.executemany(insert_query, data)
    cnx.commit()
    print("Data inserted into 'demographics' table successfully.")

In [25]:
insert_demographics_data(demographics_df)

Data inserted into 'demographics' table successfully.


In [26]:
demographics_sql_df = todf("SELECT * FROM demographics")
demographics_sql_df

,year,state,total_pop,median_age,white_pop,black_pop,male,female,high_school,bach,born_in_state,employment,inflation
0,2014,AK,736732,33,455009,23608,385582,351150,33253,85634,123013,6,236
1,2015,AK,738432,33,452472,24739,388695,349737,34616,87869,128626,6,237
2,2016,AK,741894,33,452757,21768,390573,351321,32977,90861,128699,6,240
3,2017,AK,739795,34,448081,21192,385776,354019,40036,86444,139475,6,245
4,2018,AK,737438,34,443304,24011,382952,354486,32444,89635,133565,6,251
...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,2015,WY,586107,36,492659,4704,295102,291005,30425,67108,127301,4,237
296,2016,WY,585501,37,494078,5400,298205,287296,26691,67845,133432,5,240
297,2017,WY,579315,37,486586,5154,295779,283536,27353,67312,131378,4,245
298,2018,WY,577737,38,485040,3102,292619,285118,26309,66449,133931,4,251


# elections

In [112]:
a = todf('''
SELECT A.year, B.state, 
    (MAX(B.total_votes))/(MAX(C.total_pop)) AS vote_turnout,
    SUM(A.total_raised) AS total_raised,
    SUM(A.total_spent) AS total_spent,
    SUM(A.cash_start) AS cash_start,
    SUM(A.cash_end) AS cash_end
FROM finances AS A
LEFT JOIN results AS B ON A.fec_id = B.fec_id
LEFT JOIN (
    SELECT state, (year) + 1 AS next_year, total_pop
    FROM demographics
) AS C ON B.state = C.state AND A.year = C.next_year
GROUP BY B.state, A.year
''')

In [113]:
a

,year,state,vote_turnout,total_raised,total_spent,cash_start,cash_end
0,2016,IL,0.4641,1.439186e+06,1.362855e+06,7.431950e+05,8.195260e+05
1,2016,KY,0.4825,1.620247e+07,1.829747e+07,3.253052e+06,1.158047e+06
2,2016,AL,0.4296,3.500908e+06,1.156380e+07,1.805543e+07,9.992079e+06
3,2016,MA,0.5246,6.522036e+06,3.530796e+06,3.879394e+06,6.870637e+06
4,2016,IA,0.5352,1.435565e+07,1.493494e+07,1.998427e+06,1.106562e+06
...,...,...,...,...,...,...,...
145,2020,GA,0.4664,2.692916e+08,2.531360e+08,1.977942e+06,1.813787e+07
146,2020,NV,0.3598,6.591753e+06,3.344429e+06,1.445311e+06,4.687382e+06
147,2020,LA,0.4456,7.302850e+06,4.491204e+06,3.023384e+06,5.842530e+06
148,2020,UT,0.3315,8.930188e+05,5.498722e+05,2.582317e+05,6.013782e+05


In [114]:
a.dropna(inplace=True)

In [115]:
a

,year,state,vote_turnout,total_raised,total_spent,cash_start,cash_end
0,2016,IL,0.4641,1.439186e+06,1.362855e+06,7.431950e+05,8.195260e+05
1,2016,KY,0.4825,1.620247e+07,1.829747e+07,3.253052e+06,1.158047e+06
2,2016,AL,0.4296,3.500908e+06,1.156380e+07,1.805543e+07,9.992079e+06
3,2016,MA,0.5246,6.522036e+06,3.530796e+06,3.879394e+06,6.870637e+06
4,2016,IA,0.5352,1.435565e+07,1.493494e+07,1.998427e+06,1.106562e+06
...,...,...,...,...,...,...,...
144,2020,CA,0.2813,1.775393e+05,2.618597e+05,1.323391e+05,4.801867e+04
145,2020,GA,0.4664,2.692916e+08,2.531360e+08,1.977942e+06,1.813787e+07
146,2020,NV,0.3598,6.591753e+06,3.344429e+06,1.445311e+06,4.687382e+06
147,2020,LA,0.4456,7.302850e+06,4.491204e+06,3.023384e+06,5.842530e+06


In [116]:
def create_elections_table():
    query = """
    CREATE TABLE IF NOT EXISTS elections (
        year YEAR,
        state CHAR(2),
        vote_turnout FLOAT,
        total_raised FLOAT,
        total_spent FLOAT,
        cash_start FLOAT,
        cash_end FLOAT,
        PRIMARY KEY (year, state)
    )
    """
    cur = cnx.cursor()
    cur.execute(query)
    cnx.commit()
    print("Table 'elections' has been created successfully.")

In [117]:
print('before')
sql('SHOW TABLES')
sql("DROP TABLE IF EXISTS elections")
print('table deleted')
sql('SHOW TABLES')
create_elections_table()
print('after')
sql('SHOW TABLES')

print('''

Below are the columns for that new table:
      ''')
sql("SELECT column_name FROM information_schema.columns WHERE table_name = 'elections'")

before
('demographics',)
('elections',)
('finances',)
('results',)
SUCCESS: but bro there is nothing to print 'NoneType' object is not subscriptable
table deleted
('demographics',)
('finances',)
('results',)
Table 'elections' has been created successfully.
after
('demographics',)
('elections',)
('finances',)
('results',)


Below are the columns for that new table:
      
('cash_end',)
('cash_start',)
('state',)
('total_raised',)
('total_spent',)
('vote_turnout',)
('year',)


In [125]:
def insert_elections_data(elections_df):
    insert_query = """
        INSERT INTO elections (year, state, vote_turnout, total_raised, total_spent, cash_start, cash_end)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """
    data = elections_df.values.tolist()
    cur = cnx.cursor()
    cur.executemany(insert_query, data)
    cnx.commit()
    print("Data inserted into 'elections table successfully.")

In [126]:
a.head(5)

,year,state,vote_turnout,total_raised,total_spent,cash_start,cash_end
0,2016,IL,0.4641,1.439186e+06,1.362855e+06,7.431950e+05,8.195260e+05
1,2016,KY,0.4825,1.620247e+07,1.829747e+07,3.253052e+06,1.158047e+06
2,2016,AL,0.4296,3.500908e+06,1.156380e+07,1.805543e+07,9.992079e+06
3,2016,MA,0.5246,6.522036e+06,3.530796e+06,3.879394e+06,6.870637e+06
4,2016,IA,0.5352,1.435565e+07,1.493494e+07,1.998427e+06,1.106562e+06


In [127]:
insert_elections_data(a)

Data inserted into 'elections table successfully.


In [128]:
elections_sql_df = todf("SELECT * FROM elections")
elections_sql_df

,year,state,vote_turnout,total_raised,total_spent,cash_start,cash_end
0,2016,AK,0.4802,6137850.0,7812960.0,1183380.0,353721.0
1,2016,AL,0.4296,3500910.0,11563800.0,18055400.0,9992080.0
2,2016,AR,0.4007,8123460.0,7903670.0,485392.0,742600.0
3,2016,CA,0.2839,592780.0,466186.0,2297760.0,2424360.0
4,2016,CO,0.5930,20781800.0,20219600.0,1511460.0,2073650.0
...,...,...,...,...,...,...,...
142,2020,VT,0.5029,12810400.0,14108800.0,9100700.0,7008430.0
143,2020,WA,0.4259,3437100.0,2821390.0,3262870.0,3878570.0
144,2020,WI,0.5064,5447370.0,3487120.0,554632.0,2514890.0
145,2020,WV,0.4346,8943540.0,7037640.0,3143380.0,5051060.0


In [129]:
results_df = todf("SELECT * FROM results")
results_df.head(5)

,year,state,fec_id,incumbent,first_name,last_name,party,votes,vote_share,total_votes,is_male,white,black,age
0,2016,AK,S0AK00121,0,Joe,Miller,L,90825,0.292,311441,1,1,0,48
1,2016,AR,S0AR00150,1,John,Boozman,R,661984,0.598,1107522,1,1,0,65
2,2016,CO,S0CO00211,1,Michael,Bennet,D,1370710,0.500,2743029,1,1,0,51
3,2016,CT,S0CT00177,1,Richard,Blumenthal,D,920766,0.577,1596276,1,1,0,70
4,2016,CT,S0CT00250,0,Jeffery,Russell,O,16713,0.010,1596276,1,1,0,56


In [130]:
finances_df = todf("SELECT * FROM finances")
finances_df.head(5)

,year,committee_id,fec_id,committee_name,total_raised,total_spent,cash_start,cash_end
0,2016,C00148999,S6IL00151,FRIENDS OF DICK DURBIN,1151490.0,1073760.0,741450.0,819176.0
1,2016,C00193342,S2KY00012,MCCONNELL SENATE COMMITTEE,2154060.0,1476860.0,248241.0,925444.0
2,2016,C00193623,S6AL00013,SHELBY FOR U S SENATE,3467640.0,11531400.0,18055400.0,9991640.0
3,2016,C00196774,S4MA00028,MARKEY COMMITTEE; THE,601552.0,803795.0,2227330.0,2025090.0
4,2016,C00230482,S0IA00028,GRASSLEY COMMITTEE INC,8954940.0,9787430.0,1838650.0,1006160.0


In [131]:
demographics_df = todf("SELECT * FROM demographics")
demographics_df.head(5)

,year,state,total_pop,median_age,white_pop,black_pop,male,female,high_school,bach,born_in_state,employment,inflation
0,2014,AK,736732,33,455009,23608,385582,351150,33253,85634,123013,6,236
1,2015,AK,738432,33,452472,24739,388695,349737,34616,87869,128626,6,237
2,2016,AK,741894,33,452757,21768,390573,351321,32977,90861,128699,6,240
3,2017,AK,739795,34,448081,21192,385776,354019,40036,86444,139475,6,245
4,2018,AK,737438,34,443304,24011,382952,354486,32444,89635,133565,6,251


In [132]:
elections_df = todf("SELECT * FROM elections")
elections_df.head(5)

,year,state,vote_turnout,total_raised,total_spent,cash_start,cash_end
0,2016,AK,0.4802,6137850.0,7812960.0,1183380.0,353721.0
1,2016,AL,0.4296,3500910.0,11563800.0,18055400.0,9992080.0
2,2016,AR,0.4007,8123460.0,7903670.0,485392.0,742600.0
3,2016,CA,0.2839,592780.0,466186.0,2297760.0,2424360.0
4,2016,CO,0.5930,20781800.0,20219600.0,1511460.0,2073650.0
